In [1]:
from ase.build import bulk
from pyiron_workflow_atomistics.dataclass_storage import CalcInputMinimize
from pyiron_workflow_atomistics.bulk import optimise_cubic_lattice_parameter
from pyiron_workflow_lammps.engine import LammpsEngine
from pyiron_workflow import Workflow

import os

from vacancy import get_vacancy_formation_energy

%load_ext autoreload
%autoreload 2

# Example for LAMMPS engine

In [ ]:
from pyiron_workflow import Workflow
from pyiron_workflow_atomistics.surface.utils import (
    create_surface_from_symbol,
    create_surface_slab,
    get_surface_info
)

# Create workflow
wf = Workflow("surface_study")

# Method 1: Direct surface creation from symbol
wf.fe_slab = create_surface_from_symbol(
    "Fe", 
    miller_indices=(1, 1, 0), 
    layers=4, 
    vacuum=15.0,
    crystalstructure="bcc",
    a=2.87
)

# Method 2: Create bulk first, then surface
wf.fe_bulk = create_bulk_from_symbol("Fe", crystalstructure="bcc", a=2.87)
wf.fe_slab = create_surface_slab(
    wf.fe_bulk.outputs.bulk_from_symbol.value,
    miller_indices=(1, 1, 0), 
    layers=4, 
    vacuum=15.0
)

# Run workflow
wf.run()

# Get results
fe_slab = wf.fe_slab.outputs.surface_slab.value
print(f"Fe(110) slab: {len(fe_slab)} atoms")

# Get surface information
wf.surface_info = get_surface_info(fe_slab)
wf.run()
info = wf.surface_info.outputs.surface_info.value
print(f"Surface area: {info['surface_area']:.2f} Å²")

In [ ]:
inp = CalcInputMinimize()
inp.relax_cell = False
Engine = LammpsEngine(EngineInput = inp)

Engine.working_directory = "vacancy_formation_energy"
# Adjust this line to match your LAMMPS installation
# If you are using conda-lammps or a LAMMPS binary on your system, you can use directly:
Engine.command = "lmp -in in.lmp -log minimize.log"
Engine.lammps_log_filepath = "minimize.log"
Engine.input_script_pair_style = "eam/fs"
Engine.path_to_model = os.getcwd() + "/Al-Fe.eam.fs"
# /root/github_dev/pyiron_workflow_atomistics/pyiron_workflow_atomistics
structure = bulk("Fe", a=2.828, cubic=True) 
wf1 = Workflow(Engine.working_directory, delete_existing_savefiles=True)
wf1.opt_cubic_cell = optimise_cubic_lattice_parameter(
    structure=structure,
    name="Fe",
    crystalstructure="bcc",
    calculation_engine=Engine,
    parent_working_directory = "opt_cubic_cell",
    rattle=0.1,
    strain_range=(-0.02, 0.02),
    num_points=6,
    eos_type="birchmurnaghan",
)

# Any eos_type supported by ase.eos.EquationOfState can be used,
# e.g. sjeos, taylor, murnaghan, birch, birchmurnaghan, pouriertarantola, vinet, antonschmidt, p3

Engine = LammpsEngine(EngineInput = inp)
Engine.working_directory = "vacancy_formation_energy_supercell"
# Adjust this line to match your LAMMPS installation
# If you are using conda-lammps or a LAMMPS binary on your system, you can use directly:
Engine.command = "lmp -in in.lmp -log minimize.log"
Engine.lammps_log_filepath = "minimize.log"
Engine.input_script_pair_style = "eam/fs"
Engine.path_to_model = os.getcwd() + "/Al-Fe.eam.fs"


wf1.vacancy_formation_energy = get_vacancy_formation_energy(structure=wf1.opt_cubic_cell.outputs.equil_struct,
                                                            Engine=Engine,
                                                            remove_atom_index=0,
                                                            min_dimensions=[12, 12, 12],
                                                            vacancy_engine_working_directory="vacancy_supercell",
                                                            supercell_engine_working_directory="supercell")
wf1.run()


2025-09-24 02:06:52,442 - pyiron_log - WARNING - Could not find the source code to validate calculate_vacancy_formation_energy output labels against the number of returned values -- proceeding without validation


/root/miniconda3/envs/pyiron_pypi_workflow_lammps/lib/python3.12/site-packages/pyiron_workflow_lammps/engine.py:113: RuntimeWarning: 'mode' already set to 'minimize' and also implied by EngineInput; no change.
  warnings.warn(
2025-09-24 02:06:53,385 - pyiron_log - WARNING - Directory 'vacancy_formation_energy/strain_000' already exists. Existing files may be overwritten.


current mode  minimize


2025-09-24 02:06:55,122 - pyiron_log - WARNING - Directory 'vacancy_formation_energy/strain_001' already exists. Existing files may be overwritten.
2025-09-24 02:06:56,930 - pyiron_log - WARNING - Directory 'vacancy_formation_energy/strain_002' already exists. Existing files may be overwritten.
2025-09-24 02:06:58,737 - pyiron_log - WARNING - Directory 'vacancy_formation_energy/strain_003' already exists. Existing files may be overwritten.
2025-09-24 02:07:00,478 - pyiron_log - WARNING - Directory 'vacancy_formation_energy/strain_004' already exists. Existing files may be overwritten.
2025-09-24 02:07:02,186 - pyiron_log - WARNING - Directory 'vacancy_formation_energy/strain_005' already exists. Existing files may be overwritten.
2025-09-24 02:07:03,850 - pyiron_log - WARNING - Directory 'supercell' already exists. Existing files may be overwritten.


current mode  minimize


2025-09-24 02:07:05,597 - pyiron_log - WARNING - Directory 'vacancy_supercell' already exists. Existing files may be overwritten.


current mode  minimize


{'opt_cubic_cell__a0': np.float64(2.8554036006730863),
 'opt_cubic_cell__B': np.float64(178.2689296133779),
 'opt_cubic_cell__equil_energy_per_atom': np.float64(-4.012989682254272),
 'opt_cubic_cell__equil_volume_per_atom': np.float64(11.640523523888817),
 'opt_cubic_cell__volumes': [np.float64(21.287097162601995),
  np.float64(21.81268105404494),
  np.float64(22.346845717919777),
  np.float64(22.889660634165235),
  np.float64(23.441195282719992),
  np.float64(24.001519143522813)],
 'opt_cubic_cell__structures': [Atoms(symbols='Fe2', pbc=True, cell=[[2.77144, 1.6970175625144705e-16, 1.6970175625144705e-16], [0.0, 2.77144, 1.6970175625144705e-16], [0.0, 0.0, 2.77144]]),
  Atoms(symbols='Fe2', pbc=True, cell=[[2.794064, 1.7108707671064253e-16, 1.7108707671064253e-16], [0.0, 2.794064, 1.7108707671064253e-16], [0.0, 0.0, 2.794064]]),
  Atoms(symbols='Fe2', pbc=True, cell=[[2.816688, 1.72472397169838e-16, 1.72472397169838e-16], [0.0, 2.816688, 1.72472397169838e-16], [0.0, 0.0, 2.816688]]),


In [7]:
vac_form_energy = wf1.vacancy_formation_energy.outputs.vacancy_formation_energy.value

print(f"Vacancy formation energy of Fe in Fe BCC: {vac_form_energy:.2f} eV")

Vacancy formation energy of Fe in Fe BCC: 5.74 eV
